In [8]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from dotenv import load_dotenv
import os


# --- Shared State ---
class LibraryState(TypedDict):
    question: str
    faq_answer: str
    checkout_info: str
    final_answer: str



def ClassifierAgent(state: LibraryState):
    print("ClassifierAgent ran")
    print(f"state: {state}")

    # return {}
    # return {"faq_answer": "CLASSIFIER FAQ ANSWER", "checkout_info": ""}
    return {"faq_answer": "The library opens at 9:00 AM", "checkout_info": ""}
    # return {"faq_answer": "", "checkout_info": "The Hobbit is available for checkout."}


def FAQAgent(state: LibraryState):
    print("FAQAgent ran")
    print(f"FAQAgent state", state)
    
    # return {}
    return {"faq_answer": "The library opens at 9:00 AM"}

def CheckoutAgent(state: LibraryState):
    print("CheckoutAgent ran")
    
    # return {}
    return {"checkout_info": "The Hobbit is available for checkout."}

def ResponseAgent(state: LibraryState):
    print("ResponseAgent ran")
    
    #return {"final_answer": "RESPONSE AGENT FINAL ANSWER"}
    return {"final_answer": f"FAQ: {state['faq_answer']} | Checkout: {state['checkout_info']}"}
    


# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()



In [9]:


# --- Visualize ---
print(graph.get_graph().draw_ascii())
graph.get_graph().draw_png("images/agentic_ai_library.png")
print("Graph saved as agentic_ai_library.png")

               +-----------+              
               | __start__ |              
               +-----------+              
                     *                    
                     *                    
                     *                    
            +-----------------+           
            | ClassifierAgent |           
            +-----------------+           
              ***          **             
             *               **           
           **                  **         
+---------------+           +----------+  
| CheckoutAgent |           | FAQAgent |  
+---------------+           +----------+  
              ***          **             
                 *       **               
                  **   **                 
             +---------------+            
             | ResponseAgent |            
             +---------------+            
                     *                    
                     *                    
           

In [10]:
# --- Run ---
result = graph.invoke({"question": "When does library open?"})
print("\n--- Final Answer ---")
print(result["final_answer"])


ClassifierAgent ran
state: {'question': 'When does library open?'}
CheckoutAgent ran
FAQAgent ran
FAQAgent state {'question': 'When does library open?', 'faq_answer': 'The library opens at 9:00 AM', 'checkout_info': ''}
ResponseAgent ran

--- Final Answer ---
FAQ: The library opens at 9:00 AM | Checkout: The Hobbit is available for checkout.


In [11]:
result = graph.invoke({"question": "Is The Hobbit available?"})
print("\n--- Final Answer ---")
print(result["final_answer"])


ClassifierAgent ran
state: {'question': 'Is The Hobbit available?'}
CheckoutAgent ran
FAQAgent ran
FAQAgent state {'question': 'Is The Hobbit available?', 'faq_answer': 'The library opens at 9:00 AM', 'checkout_info': ''}
ResponseAgent ran

--- Final Answer ---
FAQ: The library opens at 9:00 AM | Checkout: The Hobbit is available for checkout.
